In [1]:
# Carregando Bibliotegas e Funções
import numpy as np
from sklearn.metrics import pairwise_distances
from geopy.distance import great_circle
from grakel import graph_from_networkx, GraphKernel
import networkx as nx
from math import radians, sin, cos, sqrt, atan2
from fastdtw import fastdtw
from networkx.algorithms import isomorphism
from networkx.algorithms.similarity import optimize_graph_edit_distance
from concurrent.futures import ThreadPoolExecutor, TimeoutError
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.preprocessing import normalize, binarize
from sklearn.manifold import MDS
from scipy.spatial.distance import cdist
from scipy.spatial.distance import cosine, cityblock, jaccard
from tslearn.metrics import dtw as ts_dtw
import Levenshtein as lev

d:\Python\Python311\Lib\site-packages\tslearn\bases\bases.py:15: UserWarning: h5py not installed, hdf5 features will not be supported.
Install h5py to use hdf5 features: http://docs.h5py.org/
  warn(h5py_msg)


In [247]:
def gerar_grafos_com_coincidencia():
    G1 = nx.DiGraph()
    G1.add_node("A",label="A")
    G1.add_node("B",label="B")
    G1.add_node("C",label="C")
    G1.add_node("D",label="D")
    G1.add_node("E",label="E")
    G1.add_node("F",label="F")
    
    # Adicionar nós e arestas compartilhadas
    G1.add_edge("A", "B")
    G1.add_edge("B", "C")
    G1.add_edge("C", "D")
    
    # Adicionar nós e arestas exclusivas ao G1
    G1.add_edge("D", "E")
    G1.add_edge("E", "F")

    nx.set_node_attributes(G1, {"A": "A", "B": "B", "C": "C", "D": "D", "E":"E", "F":"F"}, "label")
    # Criar o segundo grafo
    G2 = nx.DiGraph()
    G2.add_node("A",label="A")
    G2.add_node("B",label="B")
    G2.add_node("C",label="C")
    G2.add_node("D",label="D")
    G2.add_node("G",label="G")
    G2.add_node("H",label="H")  
    G2.add_node("I",label="I")
    # Adicionar nós e arestas compartilhadas
    G2.add_edge("A", "B")
    G2.add_edge("B", "C")
    G2.add_edge("C", "D")
    
    # Adicionar nós e arestas exclusivas ao G2
    G2.add_edge("D", "G")
    G2.add_edge("G", "H")
    G2.add_edge("H", "I")
    nx.set_node_attributes(G2, {"A": "A", "B": "B", "C": "C", "D": "D", "G":"G", "H":"H","I":"I"}, "label")
    return G1, G2

In [248]:
def gerar_grafos_subgrafo():
    G1 = nx.DiGraph()
    G1.add_node("A",label="A")
    G1.add_node("B",label="B")
    G1.add_node("C",label="C")
    G1.add_node("D",label="D")
    G1.add_node("E",label="E")
    G1.add_node("F",label="F")
    
    # Adicionar nós e arestas compartilhadas
    G1.add_edge("A", "B")
    G1.add_edge("B", "C")
    G1.add_edge("C", "D")
    
    # Adicionar nós e arestas exclusivas ao G1
    G1.add_edge("D", "E")
    G1.add_edge("E", "F")

    nx.set_node_attributes(G1, {"A": "A", "B": "B", "C": "C", "D": "D", "E":"E", "F":"F"}, "label")
    # Criar o segundo grafo
    G2 = nx.DiGraph()
    G2.add_node("A",label="A")
    G2.add_node("B",label="B")
    G2.add_node("C",label="C")
    G2.add_node("D",label="D")
   
    # Adicionar nós e arestas compartilhadas
    G2.add_edge("A", "B")
    G2.add_edge("B", "C")
    G2.add_edge("C", "D")
    
    # Adicionar nós e arestas exclusivas ao G2
 
    nx.set_node_attributes(G2, {"A": "A", "B": "B", "C": "C", "D": "D"}, "label")
    return G1, G2

In [249]:
def distancia_edit2(g1, g2):
    def compute_edit_distance():
        gen = optimize_graph_edit_distance(g1, g2)
        return next(gen, float('inf'))  # se não vier nada, retorna inf

    with ThreadPoolExecutor(max_workers=1) as executor:
        future = executor.submit(compute_edit_distance)
        try:
            result = future.result(timeout=3.0)
            return result if result is not None else float('inf')
        except TimeoutError:
            print("⚠️ Timeout ao calcular distância de edição")
            return float('inf')
        except Exception as e:
            print(f"❌ Erro ao calcular distância de edição: {e}")
            return float('inf')

In [250]:
def distancia_edit(g1, g2):
    ged = nx.graph_edit_distance(g1, g2, roots=None, timeout=3.0)
    return ged

In [251]:
def distancia_edicao_grafos(g1, g2):
    """
    Calcula a distância de edição entre dois grafos.
    """
    # Converter arestas para conjuntos para facilitar a comparação
    edges_g1 = set(g1.edges())
    edges_g2 = set(g2.edges())

    # Operações de edição
    arestas_removidas = edges_g1 - edges_g2  # Arestas em g1 que não estão em g2
    arestas_adicionadas = edges_g2 - edges_g1  # Arestas em g2 que não estão em g1

    # Distância de edição é a soma das operações
    distancia = len(arestas_removidas) + len(arestas_adicionadas)

    return distancia

In [252]:
grafo1, grafo2 = gerar_grafos_com_coincidencia()
print('Grafo1', grafo1.nodes())
print('Grafo2',grafo2.nodes())
edit_distance = distancia_edit(grafo1, grafo2)
edit_distance2 = distancia_edit2(grafo1, grafo2)
edit_distance3 = distancia_edicao_grafos(grafo1, grafo2)
print(f"Distância de Edição (Método 1): {edit_distance}")
print(f"Distância de Edição (Método 2): {edit_distance2}")
print(f"Distância de Edição (Método 3): {edit_distance3}")

Grafo1 ['A', 'B', 'C', 'D', 'E', 'F']
Grafo2 ['A', 'B', 'C', 'D', 'G', 'H', 'I']
Distância de Edição (Método 1): 2.0
Distância de Edição (Método 2): 2.0
Distância de Edição (Método 3): 5


In [206]:
def jaccard_similarity(g1, g2):
    edges_g1 = set(g1.edges())
    edges_g2 = set(g2.edges())

    # Calcular a interseção e a união dos conjuntos de arestas
    intersecao = len(edges_g1.intersection(edges_g2))
    uniao = len(edges_g1.union(edges_g2))

    # Evitar divisão por zero
    if uniao == 0:
        return 0.0

    # Similaridade de Jaccard
    return intersecao / uniao

In [253]:
print('Grafo1', grafo1.nodes())
print('Grafo2',grafo2.nodes())
edit_distance = jaccard_similarity(grafo1, grafo2)
print(f"Distância de jaccard (Método 1): {edit_distance}")

Grafo1 ['A', 'B', 'C', 'D', 'E', 'F']
Grafo2 ['A', 'B', 'C', 'D', 'G', 'H', 'I']
Distância de jaccard (Método 1): 0.375


In [254]:
def graph_kernel(g1, g2, label="label"):
    # Converter grafos NetworkX para formato utilizado pelo GraKeL
    G1_grakel = list(graph_from_networkx([g1], node_labels_tag=label, edge_labels_tag=None))
    G2_grakel = list(graph_from_networkx([g2], node_labels_tag=label, edge_labels_tag=None))
    # Escolher um kernel, por exemplo, o Weisfeiler-Lehman kernel
    gk = GraphKernel(kernel=["weisfeiler_lehman", "subtree_wl"])
    # Calcular similaridade
    sim_matrix = gk.fit_transform([G1_grakel[0], G2_grakel[0]])
    return sim_matrix[0,1] 

In [255]:
print('Grafo1', grafo1.nodes())
print('Grafo2',grafo2.nodes())
edit_distance = graph_kernel(grafo1, grafo2)
print(f"Distância de kernel (Método 1): {edit_distance}")

Grafo1 ['A', 'B', 'C', 'D', 'E', 'F']
Grafo2 ['A', 'B', 'C', 'D', 'G', 'H', 'I']
Distância de kernel (Método 1): 10.0


In [267]:
def subgraph_similarity(g1, g2):
    GM = isomorphism.GraphMatcher(g1, g2)
    # Tenta encontrar uma correspondência de subgrafos
    matches = list(GM.subgraph_isomorphisms_iter())
    if not matches:
        return 0.0
    
    # Pega o melhor match
    best_match = max(matches, key=lambda m: len(m))
    match_size = len(best_match)
    
    # Normaliza pela quantidade de nós do menor grafo
    min_nodes = min(len(g1.nodes), len(g2.nodes))
    return match_size / min_nodes

def subgraph_similarity_rapida(g1, g2):
    # Interseção e união dos nós
    nodes_g1 = set(g1.nodes())
    nodes_g2 = set(g2.nodes())
    node_intersection = len(nodes_g1 & nodes_g2)
    node_union = len(nodes_g1 | nodes_g2)
    
    # Interseção e união das arestas (como tuplas de pares)
    edges_g1 = set(g1.edges())
    edges_g2 = set(g2.edges())
    edge_intersection = len(edges_g1 & edges_g2)
    edge_union = len(edges_g1 | edges_g2)

    # Similaridade baseada na média das Jaccards de nós e arestas
    node_sim = node_intersection / node_union if node_union else 0
    edge_sim = edge_intersection / edge_union if edge_union else 0

    return (node_sim + edge_sim) / 2

def similaridade_grafos(g1, g2, atributo='pos'):
    pos_g1 = nx.get_node_attributes(g1, atributo)
    pos_g2 = nx.get_node_attributes(g2, atributo)
    ponto = 0
    for node1 in pos_g1:
        coords_g1=pos_g1[node1]
        existe_ponto = False
        for node2 in pos_g2:
            coords_g2 = pos_g2[node2]
            km = great_circle(coords_g1,coords_g2).kilometers
            if km<=0.2:
                existe_ponto=True
                break
        if existe_ponto:
             ponto = ponto + 1
    return ponto / len(pos_g1) if len(pos_g1) > 0 else 0

In [268]:
grafo1, grafo2 = gerar_grafos_subgrafo()
print('Grafo1', grafo1.nodes())
print('Grafo2',grafo2.nodes())
edit_distance = subgraph_similarity(grafo1, grafo2)
edit_distance2 = subgraph_similarity_rapida(grafo1, grafo2)
edit_distance3 = similaridade_grafos(grafo1, grafo2)
print(f"Distância de subgrafo (Método 1): {edit_distance}")
print(f"Distância de subgrafo (Método 2): {edit_distance2}")
print(f"Distância de subgrafo (Método 3): {edit_distance3}")

Grafo1 ['A', 'B', 'C', 'D', 'E', 'F']
Grafo2 ['A', 'B', 'C', 'D']
Distância de subgrafo (Método 1): 1.0
Distância de subgrafo (Método 2): 0.6333333333333333
Distância de subgrafo (Método 3): 0
